In [1]:
from pyfirmata import Arduino
import serial
import time

In [4]:
class MCSArduino(Arduino):
    pin_diode = 13
    pin_TTL = 6
    pin_LED_lights = 7
    
    def __init__(self, *args, **kwargs):
        self.last_cmd = False  # False - Arduino LOW, True - Arduino HIGH
        self.is_light_off = False
        super(MCSArduino, self).__init__(*args, **kwargs)
        
    def start_or_stop(self):
        self.last_cmd = not self.last_cmd
        self.digital[MCSArduino.pin_diode].write(self.last_cmd)
        self.digital[MCSArduino.pin_TTL].write(self.last_cmd)

    def switch_light(self):
        self.is_light_off = not self.is_light_off
        self.digital[MCSArduino.pin_LED_lights].write(self.is_light_off)
        
class FakeArduino():
    def start_or_stop(self):
        print("Fake Arduino - sending a TTL pulse")
        
    def exit(self):
        print("Fake Arduino - exiting...")

In [3]:
class Feeder:
    def __init__(self, port, baudrate=9600):
        self.device = serial.Serial(port, baudrate=baudrate)
        
    def feed(self):
        self.device.write('SP200\r'.encode())
        self.device.write('LR6175\r'.encode())
        self.device.write('m\r'.encode())
        
    def exit(self):
        self.device.close()

### Testing Arduino TTL pulses

In [9]:
board = MCSArduino('COM10')  # Windows - 'COM10', Linux - '/dev/ttyACM0', check /dev/tty*

In [11]:
# check the light on-off
board.switch_light()

In [21]:
# Arduino should blink 2 times
for i in range(4):
    board.start_or_stop()
    time.sleep(2)

In [12]:
board.exit()

### Testing feeder

In [4]:
feeder = Feeder('COM8')

time.sleep(5)  # time to get inside the chamber and see that it works
feeder.feed()

feeder.exit()